<a href="https://colab.research.google.com/github/berbir12/dermis/blob/main/recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

data_path = '/content/drive/MyDrive/export_skincare.csv'

df = pd.read_csv(data_path)
print("Data loaded successfully!")
print(df.head())



df.columns = df.columns.str.strip()


skin_type_cols = ['Sensitive', 'Combination', 'Oily', 'Dry', 'Normal']
for col in skin_type_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
    else:
        print(f"Warning: Skin type column '{col}' not found in DataFrame.")


df['product_type'] = df['product_type'].fillna('')
df['brand'] = df['brand'].fillna('')
df['notable_effects'] = df['notable_effects'].fillna('')
df['skintype'] = df['skintype'].fillna('')

print("\nProcessed skin type columns:")
print(df[skin_type_cols].head())

Data loaded successfully!
   Unnamed: 0  \
0           0   
1           1   
2           2   
3           3   
4           4   

                                                                    product_href  \
0    https://www.beautyhaul.com/product/detail/bubble-free-ph-balancing-cleanser   
1  https://www.sociolla.com/face-wash/62452-ph-balancing-soothing-cleansing-foam   
2     https://www.sociolla.com/toner/15871-licorice-ph-balancing-cleansing-toner   
3              https://www.beautyhaul.com/product/detail/aquaseal-soothing-tonic   
4     https://www.sociolla.com/essence/38023-licorice-ph-balancing-essence-mist    

                                    product_name product_type    brand  \
0       ACWELL Bubble Free PH Balancing Cleanser    Face Wash  ACWELL    
1    ACWELL pH Balancing Soothing Cleansing Foam    Face Wash  ACWELL    
2  Acwell Licorice pH Balancing Cleansing Toner         Toner  ACWELL    
3                ACWELL Aquaseal Soothing Tonic         Toner  ACWELL 

In [44]:
def recommend_by_skin_type(user_skin_type, dataframe, top_n=10):
    """
    Recommends products based on a user's specified skin type.

    Args:
        user_skin_type (str): The skin type of the user (e.g., 'Oily', 'Dry', 'Normal', 'Sensitive', 'Combination').
        dataframe (pd.DataFrame): The DataFrame containing product information.
        top_n (int): The number of top recommendations to return.

    Returns:
        pd.DataFrame: A DataFrame of recommended products.
    """
    user_skin_type = user_skin_type.lower()

    skin_type_column_map = {
        'sensitive': 'Sensitive',
        'combination': 'Combination',
        'oily': 'Oily',
        'dry': 'Dry',
        'normal': 'Normal'
    }

    if user_skin_type not in skin_type_column_map:
        print(f"Error: '{user_skin_type}' is not a recognized skin type. Please choose from: {list(skin_type_column_map.keys())}")
        return pd.DataFrame()

    target_column = skin_type_column_map[user_skin_type]

    if target_column not in dataframe.columns:
        print(f"Error: Column '{target_column}' not found in the DataFrame. Please check column names.")
        return pd.DataFrame()


    filtered_products = dataframe[
        (dataframe[target_column] == 1) |
        (dataframe['skintype'].str.contains(user_skin_type, case=False, na=False))
    ].copy()

    if filtered_products.empty:
        print(f"No products found for '{user_skin_type}' skin type.")
        return pd.DataFrame()


    return filtered_products.head(top_n)

In [45]:
import os
from fastai.vision.all import *

model_path = '/content/drive/MyDrive/export (1).pkl'

if os.path.exists(model_path):
    try:
        skin_type_model = load_learner(model_path)
        print(f"Model loaded successfully from {model_path} using fastai")
    except Exception as e:
        print(f"Error loading fastai model from {model_path}: {e}")
else:
    print(f"Model file not found at {model_path}")

/usr/local/lib/python3.11/dist-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


Model loaded successfully from /content/drive/MyDrive/export (1).pkl using fastai


In [46]:
from fastai.vision.all import *
import requests
from io import BytesIO
import os

image_url = "https://www.fast.ai/images/logo-small.png"

try:
    response = requests.get(image_url)
    response.raise_for_status()

    img_bytes = BytesIO(response.content)

    img = PILImage.create(img_bytes)
    print("Image loaded successfully from URL using fastai.")

    img_for_classification = img

except requests.exceptions.RequestException as e:
    print(f"Error downloading image from URL: {e}")
    img_for_classification = None
except Exception as e:
    print(f"Error loading image with fastai after download: {e}")
    img_for_classification = None

Error fetching image from URL: 403 Client Error: Forbidden. Please comply with the User-Agent policy: https://meta.wikimedia.org/wiki/User-Agent_policy for url: https://upload.wikimedia.org/wikipedia/commons/thumb/a/a0/Andromeda_Galaxy_300M_pixels.jpg/1200px-Andromeda_Galaxy_300M_pixels.jpg
Image loaded successfully from local file 'face.jpg'.
Image resized to (224, 224).
Image converted to NumPy array with shape: (224, 224, 3)
Image pixel values normalized to [0, 1].
Image dimensions expanded to: (1, 224, 224, 3)


In [60]:
if 'skin_type_model' in locals() and skin_type_model is not None and \
   'img_for_classification' in locals() and img_for_classification is not None:

    predictions = skin_type_model.predict(img_for_classification)

    print("Raw prediction results:")
    print(predictions)

else:
    if 'skin_type_model' not in locals() or skin_type_model is None:
        print("Error: 'skin_type_model' not found or is None. Model loading failed.")
    if 'img_for_classification' not in locals() or img_for_classification is None:
        print("Error: 'img_for_classification' not found or is None. Image preparation failed.")
    print("Skipping classification step.")

Raw prediction results:
('oily', tensor(2), tensor([8.2780e-06, 1.2016e-08, 9.9999e-01]))


In [61]:
if 'predictions' in locals() and predictions is not None:

    try:
        predicted_class = predictions[0]

        predicted_skin_type = str(predicted_class)

        print(f"Predicted Skin Type: {predicted_skin_type}")


    except IndexError:
        print(f"Error: Unexpected format for predictions. Expected a tuple with at least one element, but got: {predictions}")
        predicted_skin_type = None
    except Exception as e:
        print(f"An error occurred while processing predictions: {e}")
        predicted_skin_type = None


else:
    print("Classification results ('predictions') are not available. Please run the classification step first.")
    predicted_skin_type = None

Predicted Skin Type: oily


In [62]:
if 'predicted_skin_type' in locals() and predicted_skin_type is not None:
    print(f"Predicted skin type available: {predicted_skin_type}")
    recommended_products = recommend_by_skin_type(predicted_skin_type, df)

    if not recommended_products.empty:
        print("\nRecommended products:")
        display(recommended_products[['product_name', 'product_type', 'brand', 'skintype', 'notable_effects']])
    else:
        print(f"No recommendations found for {predicted_skin_type} skin type.")

else:
    print("Predicted skin type is not available. Cannot generate recommendations.")

Predicted skin type available: oily

Recommended products:


,product_name,product_type,brand,skintype,notable_effects
0,ACWELL Bubble Free PH Balancing Cleanser,Face Wash,ACWELL,Oily,"Acne-Free, Pore-Care, Brightening, Anti-Aging"
2,Acwell Licorice pH Balancing Cleansing Toner,Toner,ACWELL,"Normal, Dry, Oily, Combination, Sensitive","Soothing, Balancing"
3,ACWELL Aquaseal Soothing Tonic,Toner,ACWELL,Oily,"Acne-Free, Pore-Care, Brightening, Anti-Aging"
5,Licorice pH Balancing Advance Serum,Serum,ACWELL,"Normal, Dry, Oily, Combination","Moisturizing, Brightening, Anti-Aging"
6,Licorice pH Balancing Essential Cream,Moisturizer,ACWELL,"Normal, Dry, Oily, Combination, Sensitive","Moisturizing, Brightening"
8,AHC Peony Bright Clearing Toner,Toner,AHC,Oily,"Pore-Care, Brightening, Anti-Aging"
9,AHC Hyaluronic Toner,Toner,AHC,Oily,Anti-Aging
10,AHC Peony Bright Luminous Serum,Serum,AHC,Oily,"Brightening, Anti-Aging, UV-Protection"
11,AIZEN SunOrion UV Sunscreen,Sunscreen,AIZEN,"Dry, Oily, Sensitive","Anti-Aging, UV-Protection"
13,AIZEN Whitifique Face Cream,Moisturizer,AIZEN,Oily,"Brightening, Anti-Aging, UV-Protection"


In [59]:
if 'recommended_products' in locals() and not recommended_products.empty:

    print("Showing recommended products:")
    display(recommended_products)
else:
    print("No recommendations could be displayed.")

Showing recommended products:


,Unnamed: 0,product_href,product_name,product_type,brand,notable_effects,skintype,price,description,picture_src,labels,Sensitive,Combination,Oily,Dry,Normal
1,1,https://www.sociolla.com/face-wash/62452-ph-balancing-soothing-cleansing-foam,ACWELL pH Balancing Soothing Cleansing Foam,Face Wash,ACWELL,"Soothing, Balancing","Normal, Dry, Combination",Rp 181.800,"Membersihkan dan menenangkan kulit sensitif dengan gelembung padat. Membersihkan kotoran dan sebum dengan lembut, menjaga kulit tetap lembab",https://images.soco.id/8f08ced0-344d-41f4-a15e-9e45c898f92d-.jpg,127,0,1,0,1,1
2,2,https://www.sociolla.com/toner/15871-licorice-ph-balancing-cleansing-toner,Acwell Licorice pH Balancing Cleansing Toner,Toner,ACWELL,"Soothing, Balancing","Normal, Dry, Oily, Combination, Sensitive",Rp 149.000,"Mengangkat sisa kotoran, debu, dan make up sekaligus menyeimbangkan pH alami kulit!","https://www.soco.id/cdn-cgi/image/w=73,format=auto,dpr=1.45/https://images.soco.id/19667957711-1583664703095.png",127,1,1,1,1,1
4,4,https://www.sociolla.com/essence/38023-licorice-ph-balancing-essence-mist,Licorice pH Balancing Essence Mist,Toner,ACWELL,"Brightening, Soothing","Normal, Dry",Rp 194.650,"Essens mist dengan kelembapan tinggi yang memberi efek menyejukkan, mencerahkan, dan menenangkan kulit. Mengandung 10% air licorice, ekstrak mugwort, dan centella asiatica.","https://www.sociolla.com/cdn-cgi/image/w=425,format=auto,dpr=1.45/https://images.soco.id/f60bfbd8-15e7-4d06-9575-f635e3d9d19a-image-0-1610606189420",40,0,0,0,1,1
5,5,https://www.sociolla.com/face-serum/54021-licorice-ph-balancing-advance-serum,Licorice pH Balancing Advance Serum,Serum,ACWELL,"Moisturizing, Brightening, Anti-Aging","Normal, Dry, Oily, Combination",Rp 187.000,Serum wajah yang mengandung ekstrak akar licorice untuk melembapkan serta mencerahkan dan meratakan warna kulit. Kaya akan antioksidan dan menutrisi kulit.,"https://www.sociolla.com/cdn-cgi/image/w=425,format=auto,dpr=1.45/https://images.soco.id/b6bbd6a7-56d5-4b89-9e95-d69149c87f7f-image-0-1610606096802",88,0,1,1,1,1
6,6,https://www.sociolla.com/face-cream-lotion/71655-licorice-ph-balancing-essential-cream,Licorice pH Balancing Essential Cream,Moisturizer,ACWELL,"Moisturizing, Brightening","Normal, Dry, Oily, Combination, Sensitive",Rp 304.000,"Moisturize dengan kandungan Licorice Roots yang mencerahkan dan memberikan kelembapan pada kulit, menggunakan bahan-bahan natural, kirm wajah ini aman untuk kulit sensitif","https://www.sociolla.com/cdn-cgi/image/w=244,format=auto,dpr=1.45/https://images.soco.id/83c221f3-90b2-41bd-9f50-10c721eb44ca-.jpg",87,1,1,1,1,1
7,7,https://www.sociolla.com/face-cream-lotion/62450-real-aqua-balancing-cream,Real Aqua Balancing Cream,Moisturizer,ACWELL,"Hydrating, Balancing, Moisturizing","Normal, Dry, Combination",Rp 306.000,Moisturizer ringan yang melembapkan & mengembalikan keseimbangan kulit yang dehidrasi. Krim ini mengunci kelembapan di skin barrier dengan N4 Complex yang dipatenkan. Mengandung licorice root extract dan Jeju Brussels sprouts yang menenangkan dan mencerahkan.,"https://www.sociolla.com/cdn-cgi/image/w=244,format=auto,dpr=1.45/https://images.soco.id/49975d1d-fed4-43ef-befb-0abcaf975728-.jpg",49,0,1,0,1,1
11,11,https://www.beautyhaul.com/product/detail/sunorion-uv-sunscreen,AIZEN SunOrion UV Sunscreen,Sunscreen,AIZEN,"Anti-Aging, UV-Protection","Dry, Oily, Sensitive",Rp 139.000,"Aizen SunOrion UV Sunscreen adalah tabir surya yang diformulasikan untuk memberikan proteksi maksimal dari sinar UVA yang merupakan penyebab utama penuaan dini dan kanker kulit, serta dari sinar UVB yang menyebabkan kulit gelap dan terbakar.",https://www.beautyhaul.com/assets/uploads/products/thumbs/800x800/Aizen_SunOrion_UV_Sunscreen.png,24,1,0,1,1,0
14,14,https://www.sociolla.com/sunscreen/28376-whitening-uv-sunscreen-gel,ANESSA Whitening UV Sunscreen Gel,Sunscreen,ANESSA,"Hydrating, Moisturizing","Normal, Dry, Oily, Combination, Sensitive",Rp 399.000,"Anessa Whitening UV Sunscreen Skincare Gel adala

In [ ]:
!pip install fastai